# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities_ai.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tolanaro,-25.0319,46.9987,296.19,83,20,9.26,MG,1726683315
1,1,waitangi,-43.9535,-176.5597,281.38,88,96,3.58,NZ,1726683316
2,2,adamstown,-25.0660,-130.1015,293.64,82,10,2.50,PN,1726683317
3,3,arkhangel'sk,64.5401,40.5433,285.21,100,75,2.00,RU,1726683318
4,4,ilulissat,69.2167,-51.1000,276.16,81,75,0.00,GL,1726683319


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',  
    geo=True,  
    size='Humidity', 
    color='Humidity', 
    hover_cols=['City', 'Max Temp', 'Humidity'], 
    tiles='OSM',  
    title='City Humidity Map',
    frame_width = 800,
    frame_height = 600,
    scale = 0.5,
)
    
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values

idealweather_city_data_df = city_data_df[(city_data_df['Humidity'] >= 30) & (city_data_df['Humidity'] <= 50)]

# Drop any rows with null values
idealweather_city_data_df_clean = idealweather_city_data_df.dropna()

# Display sample data
idealweather_city_data_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,reyes,-14.3167,-67.3833,309.13,44,4,4.12,BO,1726683321
8,8,elk city,35.4120,-99.4043,305.14,48,0,8.23,US,1726683325
14,14,puerto natales,-51.7236,-72.4875,280.40,39,0,7.20,CL,1726683332
22,22,ushuaia,-54.8000,-68.3000,278.96,45,40,6.69,AR,1726683341
33,33,waddan,29.1614,16.1390,307.34,32,33,7.36,LY,1726683353
50,50,lively,46.4334,-81.1498,299.27,41,3,2.13,CA,1726683372
76,76,baker city,44.7749,-117.8344,289.99,48,75,0.00,US,1726683402
81,81,liberty,31.8335,-81.5004,303.30,35,75,2.57,US,1726683407
84,84,alice springs,-23.7000,133.8833,282.90,34,0,0.00,AU,1726683165
101,101,zarichne,51.8126,26.1290,290.72,44,65,3.19,UA,1726683429


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealweather_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(20)

C:\Users\anvin\AppData\Local\Temp\ipykernel_25824\1282283278.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
6,reyes,BO,-14.3167,-67.3833,44,
8,elk city,US,35.4120,-99.4043,48,
14,puerto natales,CL,-51.7236,-72.4875,39,
22,ushuaia,AR,-54.8000,-68.3000,45,
33,waddan,LY,29.1614,16.1390,32,
50,lively,CA,46.4334,-81.1498,41,
76,baker city,US,44.7749,-117.8344,48,
81,liberty,US,31.8335,-81.5004,35,
84,alice springs,AU,-23.7000,133.8833,34,
101,zarichne,UA,51.8126,26.1290,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel

radius = 10000
params = {
    "categories":"airport",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
  
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
reyes - nearest hotel: Aeropuerto de Reyes
elk city - nearest hotel: Elk City Regional Business Airport
puerto natales - nearest hotel: Aeropuerto Teniente Julio Gallardo
ushuaia - nearest hotel: Aeroclub Ushuaia
waddan - nearest hotel: مطار ودان
lively - nearest hotel: No hotel found
baker city - nearest hotel: Baker Municipal Airport
liberty - nearest hotel: Wright Army Airfield (Fort Stewart)/Midcoast Regional Airport
alice springs - nearest hotel: No hotel found
zarichne - nearest hotel: No hotel found
andrews - nearest hotel: Andrews County Airport
tanout - nearest hotel: No hotel found
rio gallegos - nearest hotel: Aeropuerto Internacional Piloto Civil Norberto Fernández
volodarskiy - nearest hotel: No hotel found
queenstown - nearest hotel: Queenstown Airport
ribeira grande - nearest hotel: Aeroporto da Horta
desaguadero - nearest hotel: No hotel found
thunder bay - nearest hotel: Thunder Bay International Airport
green river - nearest hotel: Greater Green 

,City,Country,Lat,Lng,Humidity,Hotel Name
6,reyes,BO,-14.3167,-67.3833,44,Aeropuerto de Reyes
8,elk city,US,35.4120,-99.4043,48,Elk City Regional Business Airport
14,puerto natales,CL,-51.7236,-72.4875,39,Aeropuerto Teniente Julio Gallardo
22,ushuaia,AR,-54.8000,-68.3000,45,Aeroclub Ushuaia
33,waddan,LY,29.1614,16.1390,32,مطار ودان
...,...,...,...,...,...,...
578,mto wa mbu,TZ,-3.3500,35.8500,50,Lake Manyara Airport
583,kiama,AU,-34.6833,150.8667,41,No hotel found
593,hastings,GB,50.8552,0.5729,42,No hotel found
601,tres arroyos,AR,-38.3739,-60.2798,34,Aeropuerto Tres Arroyos


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    'Lng', 'Lat',  
    geo=True,  
    size='Humidity', 
    color='Humidity', 
    hover_cols=['City', 'Country', 'Hotel Name'], 
    tiles='OSM',  
    title='City Locations with Hotel Information',
    frame_width = 800,
    frame_height = 600,
    scale = 0.5,
)
 
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)